In [35]:
pip install psycopg2-binary pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [36]:
import psycopg2 as ps
import pandas as pd

def exec_sql(p_sql):
    # Fetch variables
    USER = 'postgres.oqwkbszzvzemgqyyebuw'
    PASSWORD = '8FCX8HZ2r1PBYBJy'
    HOST = 'aws-1-eu-west-1.pooler.supabase.com'
    PORT = 6543
    DBNAME = 'postgres'

    # Connect to the database
    try:
        connection = ps.connect(
            user=USER,
            password=PASSWORD,
            host=HOST,
            port=PORT,
            dbname=DBNAME
        )
        print("Connection successful!")

        # Create a cursor to execute SQL queries
        cursor = connection.cursor()

        # Example query
        cursor.execute(p_sql)
        draw_result(cursor)

        # Close the cursor and connection
        cursor.close()
        connection.close()
        print("Connection closed.")

    except Exception as e:
        print(f"Failed to connect: {e}")


def draw_result(cursor):
    result = cursor.fetchall()
    fields = [i[0] for i in cursor.description]
    rows = [list(i) for i in result]
    df = pd.DataFrame(rows, columns=fields)
    display(df)


def query_get_sysdate():
    return "SELECT NOW() \"Текущее время\";"


exec_sql(query_get_sysdate())

Connection successful!


,Текущее время
0,2025-11-18 18:53:46.101144+00:00


Connection closed.


### Задача 1: Загрузка и проверка данных

#### Загрузите таблицу из CSV-файла
**Посмотрите, сколько всего клиентов и столбцов в таблице**

14 столбцов, 10000 записей. Если принять во внимание только персональные данные клиентов такие как фамилия, пол, страна проживания, возраст, то количество уникальных записей сокращается до 9759.

**Проверьте, есть ли пустые ячейки или повторяющиеся клиенты**

Есть повторяющиеся клиенты, пустые ячейки присутствуют в столбце Tenure.

**Удалите ошибки, если они есть.**

In [37]:
def query_get_count_coulumns_churn():
    return ("SELECT COUNT(*) \"Кол-во столбцов churn\" "
            "  FROM  information_schema.columns "
            " WHERE table_name = 'churn';")

exec_sql(query_get_count_coulumns_churn())

Connection successful!


,Кол-во столбцов churn
0,14


Connection closed.


In [38]:
def query_get_count_rows_churn():
    return ("SELECT COUNT(*) \"Кол-во строк churn\" "
            "  FROM  churn;")

exec_sql(query_get_count_rows_churn())

Connection successful!


,Кол-во строк churn
0,10000


Connection closed.


In [39]:
def query_get_count_uniq_customers_churn():
    return ("SELECT COUNT(DISTINCT (\"Surname\", \"Gender\", \"Geography\", \"Age\")) \"Кол-во уникальных клиентов\" "
            "  FROM  churn;")

exec_sql(query_get_count_uniq_customers_churn())

Connection successful!


,Кол-во уникальных клиентов
0,9759


Connection closed.


### Задача 2: Анализ данных через SQL-запросы
#### 2.1 Сколько клиентов в каждой стране? Подсчитайте количество клиентов в каждой стране.

In [40]:
def query_get_count_customers_by_country():
    return ("SELECT "
            "   \"Geography\" \"Страна\", "
            "   count(1) \"Кол-во клиентов\" "
            "  FROM churn "
            " GROUP BY 1"
            " ORDER BY 1;")

exec_sql(query_get_count_customers_by_country())

Connection successful!


,Страна,Кол-во клиентов
0,France,5014
1,Germany,2509
2,Spain,2477


Connection closed.


**Найдите средний баланс клиентов в каждой стране**

In [41]:
def query_get_avg_balance_by_country():
    return ("SELECT "
            "   Geography \"Страна\"",
            "   ROUND(AVG(\"Balance\")::NUMERIC, 2) \"Средний баланс\""
            "  FROM churn"
            " GROUP BY 1"
            " ORDER BY 1;")

exec_sql(query_get_count_customers_by_country())

Connection successful!


,Страна,Кол-во клиентов
0,France,5014
1,Germany,2509
2,Spain,2477


Connection closed.


#### 2.2 Какие клиенты самые активные?
**Разделите клиентов на возрастные группы: молодые (18-30), средний возраст (31-45), зрелые (46-60), пожилые (60+)
Посчитайте, сколько процентов активных клиентов в каждой группе**

In [42]:
def query_most_active_customers():
    return (
        "WITH "
          "active_members AS ("
            "SELECT "
              "\"IsActiveMember\", "
              "CASE "
                "WHEN \"IsActiveMember\" = 0 THEN NULL "
              "ELSE 1 "
              "END active,"
              "CASE "
                "WHEN \"Age\" BETWEEN 18 AND 30  THEN '18-30' "
                "WHEN \"Age\" BETWEEN 31 AND 45  THEN '31-45' "
                "WHEN \"Age\" BETWEEN 46 AND 60  THEN '46-60' "
              "ELSE '60+' "
              "END age_group "
            "FROM "
              "churn "
          "),"
          "count_active_members AS ( "
            "SELECT "
              "age_group, "
              "count(active) active, "
              "count(\"IsActiveMember\") total_users "
            "FROM "
              "active_members "
            "GROUP BY 1 "
          ") "
        "SELECT "
          "age_group \"Возрастная группа\", "
          "ROUND(active * 100.0 / total_users, 2) \"Процент активных клиентов\" "
        "FROM count_active_members "
        "ORDER BY 1;")

exec_sql(query_most_active_customers())

Connection successful!


,Возрастная группа,Процент активных клиентов
0,18-30,51.12
1,31-45,49.15
2,46-60,52.22
3,60+,80.82


Connection closed.


#### 2.3 Сколькими продуктами пользуются клиенты?

**Узнайте, у скольких клиентов 1, 2, 3 или 4 продукта
Найдите средний баланс для клиентов с разным количеством продуктов**

In [43]:
def query_get_count_probucts_avg_balance():
    return ("SELECT "
                "\"NumOfProducts\" \"Кол-во продуктов\","
                "COUNT(1) \"Кол-во клиентов\","
                "ROUND(AVG(\"Balance\")::NUMERIC, 2) \"Средний баланс\""
            " FROM churn "
            "GROUP BY 1 "
            "ORDER BY 1;")

exec_sql(query_get_count_probucts_avg_balance())

Connection successful!


,Кол-во продуктов,Кол-во клиентов,Средний баланс
0,1,5084,98551.87
1,2,4590,51879.15
2,3,266,75458.33
3,4,60,93733.14


Connection closed.




```sql

```




#### Задача 3: Создание графиков
**3.1 Откуда клиенты?**
Круговая диаграмма: покажите долю клиентов из каждой страны
Столбчатая диаграмма: покажите распределение по возрасту


**3.2 Кто богаче?**
Точечный график: покажите связь между возрастом и балансом


**3.3 Кто активнее?**
Столбчатая диаграмма: процент активных клиентов в каждой стране
Сравнительная диаграмма: средний баланс активных и неактивных клиентов
Задача 4: Напишите выводы о проведенном исследовании


#### Задача 4: Напишите выводы о проведенном исследовании